In [1]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a"), UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73"), UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

In [2]:
#type(db): pymongo.cursor.Cursor
db = edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(10)

db_ls = list(db)
type(db_ls)

list

In [ ]:
#type: list of emission.storage.timeseries.builtin_timeseries.BuiltinTimeSeries
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

#time query specification
dt1 = pydt.datetime(2016, 7, 27, 17, 0)
dt2 = pydt.datetime(2016, 7, 28, 17, 30)
ts1 = int(dt1.strftime("%s"))
ts2 = int(dt2.strftime("%s"))

time_range = estt.TimeQuery("metadata.write_ts", ts1, ts2)

#BuiltinTimeSeries.get_data_df(key, time_query = None, geo_query = None, extra_query_list=None) 
#return type: list of DataFrame objects (regarding state info)
iphone_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), iphone_ts)
android_state_df_list = map(lambda t:t.get_data_df("statemachine/transition", time_range), android_ts)

iphone_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), iphone_ts)
android_battery_df_list = map(lambda t:t.get_data_df("background/battery", time_range), android_ts)

iphone_location_df_list = map(lambda t:t.get_data_df("background/location", time_range), iphone_ts)
android_location_df_list = map(lambda t:t.get_data_df("background/location", time_range), android_ts)
android_filtered_location_df_list = map(lambda t:t.get_data_df("background/filtered_location", time_range), android_ts)

In [ ]:
test_a = android_battery_df_list[2]
test_a

In [ ]:
# testing whether location tracking is enabled 
#test_a = iphone_location_df_list[1]
#test_a['fmt_time']
#test_a

In [ ]:
_iphone_config_df_list = map(lambda t:t.get_data_df("config/sensor_config", time_range), iphone_ts)
_android_config_df_list = map(lambda t:t.get_data_df("config/sensor_config", time_range), android_ts)

ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)

iphone_config_df_list=[]
for i, df in enumerate(_iphone_config_df_list):
    if len(df) != 0:
        df['config_time'] = df['metadata_write_ts'].apply(ts_to_dt)
        #df['ts'] = df['metadata_write_ts']
        iphone_config_df_list.append(df)
    else:
        iphone_config_df_list.append(None)
        print "no config log for ios" + str(i+1)

android_config_df_list=[]
for i, df in enumerate(_android_config_df_list):
    if len(df) != 0:
        df['config_time'] = df['metadata_write_ts'].apply(ts_to_dt)
        #df['ts'] = df['metadata_write_ts'] # <class 'pandas.core.series.Series'>
        android_config_df_list.append(df)
    else:
        android_config_df_list.append(None)
        print "no config log for android" + str(i+1)

In [8]:
def config_changed(df):
    changed_lst = ['Null']
    temp = df.copy()
    temp = temp.drop(['_id', 'metadata_write_ts', 'config_time'], 1)
    #print temp
    #temp.reset_index(inplace=True)
    for i in range(1, len(temp)):
        #print(temp.iloc[i]==temp.iloc[i-1])
        if temp.iloc[i].equals(temp.iloc[i-1]):
            changed_lst.append('0')
        else:
            changed_lst.append('1')
    return changed_lst

In [9]:
iphone_config_change_df_list=[]
for index, df in enumerate(iphone_config_df_list):
    if df is None:
        #no config log found 
        iphone_config_change_df_list.append(None)
    else:
        config_series = config_changed(df)
        df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
        iphone_config_change_df_list.append(df)
    
android_config_change_df_list=[]
for index, df in enumerate(android_config_df_list):
    if df is None:
        #no config log found 
        android_config_change_df_list.append(None)
    else:
        config_series = config_changed(df)
        df.loc[:, 'config_changed'] = pd.Series(config_series, index=df.index)
        android_config_change_df_list.append(df)

In [10]:
# testing if configuration is changed through config data
ios1 = iphone_config_change_df_list[0]
ios1
android1 = android_config_change_df_list[0]
android1

,_id,accuracy,accuracy_threshold,android_geofence_responsiveness,filter_distance,filter_time,geofence_radius,ios_use_remote_push_for_sync,ios_use_visit_notifications_for_detection,is_duty_cycling,metadata_write_ts,simulate_user_interaction,trip_end_stationary_mins,config_time,config_changed
0,57a455ac88f6632cbc4aad26,100,200,5000,-1,1000,100,False,False,False,1.470381e+09,False,5,2016-08-05 00:14:32.881,Null


In [16]:
#checking Android data...
android = 4
n = android-1

a_s = android_state_df_list[n][['metadata_write_ts', 'fmt_time', 'curr_state', 'transition']]
a_s.rename(columns = {'fmt_time':'state_time'}, inplace = True)

a_b = android_battery_df_list[n][['metadata_write_ts', 'fmt_time', 'battery_level_pct', 'battery_status']]
a_b.rename(columns = {'fmt_time':'battery_time'}, inplace = True)

a_c = android_config_change_df_list[n]

if False:
#if a_c is not None: 
    a_sc = pd.merge(a_s, a_c, on='metadata_write_ts', how='outer')
    a_sc.sort_values('metadata_write_ts', ascending=True, inplace=True)
    #ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
    #a_sc['dt']=a_sc['metadata_write_ts'].apply(ts_to_dt)

    #print len(a_combined)
    a_sc.fillna('-')[['state_time','config_time', 'curr_state', 'transition', 'config_changed']]

    a_all = pd.merge(a_sc, a_b, on='metadata_write_ts', how='outer')
    a_all.sort_values('metadata_write_ts', ascending=True, inplace=True)
    a_all.fillna('-')[['state_time','config_time', 'battery_time','curr_state', 'transition', 'config_changed', 'battery_level_pct', 'battery_status']]

a_sb = pd.merge(a_s, a_b, on='metadata_write_ts', how='outer')
a_sb.sort_values('metadata_write_ts', ascending=True, inplace=True)
a_sb.fillna('-')[['state_time', 'battery_time','curr_state', 'transition', 'battery_level_pct', 'battery_status']]

#states
#a_s[a_s.curr_state!=3]
a_s

#a_c

#a_sb

,metadata_write_ts,state_time,curr_state,transition
0,1.470381e+09,2016-08-05T00:08:13.716000-07:00,1,3
1,1.470381e+09,2016-08-05T00:15:55.067000-07:00,3,3
2,1.470381e+09,2016-08-05T00:15:56.096000-07:00,3,17
3,1.470381e+09,2016-08-05T00:15:56.205000-07:00,0,0
4,1.470381e+09,2016-08-05T00:15:57.383000-07:00,2,17


In [12]:
#checking iOS data...
iphone = 4
n = iphone-1
ios_s = iphone_state_df_list[n][['metadata_write_ts', 'fmt_time', 'curr_state', 'transition']]
ios_s.rename(columns = {'fmt_time':'state_time'}, inplace = True)

ios_b = iphone_battery_df_list[n][['metadata_write_ts', 'fmt_time', 'battery_level_pct', 'battery_status']]
ios_b.rename(columns = {'fmt_time':'battery_time'}, inplace = True)

ios_c = iphone_config_change_df_list[n]

if ios_c is not None:
    ios_sc = pd.merge(ios_s, ios_c, on='metadata_write_ts', how='outer')
    ios_sc.sort_values('metadata_write_ts', ascending=True, inplace=True)
    #ts_to_dt = lambda t: pydt.datetime.fromtimestamp(t)
    #ios_sc['dt']=ios_sc['metadata_write_ts'].apply(ts_to_dt)

    #print len(ios_sc)
    #ios_sc.fillna('-')[['state_time','config_time','curr_state', 'transition', 'config_changed', 'ios_use_visit_notifications_for_detection']]

# transition 7&16 are for RECEIVED_SILENT_PUSH and DATA_PUSHED
ios_s_new = ios_s[(ios_s.transition!=7) & (ios_s.transition!=16)]
ios_sb = pd.merge(ios_s_new, ios_b, on='metadata_write_ts', how='outer')
ios_sb.sort_values('metadata_write_ts', ascending=True, inplace=True)
ios_sb.fillna('-')[['state_time', 'battery_time','curr_state', 'transition', 'battery_level_pct', 'battery_status']]
#ios_s_new

#states+configs combined
#ios_sc.fillna('-')[['state_time', 'config_time','curr_state', 'transition', 'config_changed']][(ios_sc.transition!=7) & (ios_sc.transition!=16)]

#configs 
#ios_c[['config_time', 'simulate_user_interaction']]
#ios_c
#battery
ios_b

#states
#ios_s

,metadata_write_ts,battery_time,battery_level_pct,battery_status
0,1.470380e+09,2016-08-05T00:00:03.163342-07:00,65.0,1
1,1.470380e+09,2016-08-05T00:00:03.618214-07:00,65.0,1
2,1.470384e+09,2016-08-05T01:00:06.972722-07:00,45.0,1
3,1.470384e+09,2016-08-05T01:00:17.141665-07:00,44.0,1
4,1.470384e+09,2016-08-05T01:00:27.714817-07:00,44.0,1
5,1.470384e+09,2016-08-05T01:00:38.607170-07:00,44.0,1
6,1.470386e+09,2016-08-05T01:30:06.746555-07:00,33.0,1
7,1.470388e+09,2016-08-05T02:00:14.421917-07:00,25.0,1
8,1.470388e+09,2016-08-05T02:00:14.845054-07:00,25.0,1
9,1.470388e+09,2016-08-05T02:00:38.010550-07:00,25.0,1


In [ ]:
str = "2000/10/12"
a= str.split('/')
print a[0]
print a[1]
print a[2]